In [9]:
import requests
from xml.dom.minidom import parseString  # fixed import
import json
import pandas as pd
from collections import Counter

In [10]:
# Define the API from which you will get a shuffled card deck
shuffled_deck = "https://deckofcardsapi.com/api/deck/new/shuffle/?deck_count=1"

# Obtain a shuffled card deck
response = requests.get(shuffled_deck)
# Get the deck ID
deck_id = response.json()["deck_id"]

# Define the number of cards that you will draw from your card deck
number_of_cards = 5
# Define the API that you will use to draw cards
card_draw = f"https://deckofcardsapi.com/api/deck/{deck_id}/draw/?count={number_of_cards}"

# Draw the cards
response = requests.get(card_draw)
# Convert the received information into a json file
cards = response.json() 

# Resuffle the deck if necessary
if cards["remaining"] == 0:

    # Use this code block to the reshuffle the card deck
    deck_id = response.json()["deck_id"]
    shuffle_deck = f"https://deckofcardsapi.com/api/deck/{deck_id}/shuffle/"
    response = requests.get(shuffle_deck)

In [11]:
# Define the tags that you want to know about the cards that you have drawn
retrieveTags = ['value', 'suit']

cards_retrieved = []

# Loop through each card
for card in cards["cards"]:
    dataList = [] 
    
    # Loop through each tag to get the info about your cards
    for tag in retrieveTags:

        dataList.append(card[tag])
    
    cards_retrieved.append(dataList)

cards_retrieved

[['ACE', 'CLUBS'],
 ['JACK', 'DIAMONDS'],
 ['5', 'CLUBS'],
 ['ACE', 'SPADES'],
 ['KING', 'SPADES']]

Refer to the official documentation ([see official documentation](https://docs.python.org/3/library/collections.html#collections.Counter)) on how to use Counter for counting hashable objects from Collections python package.

In [12]:
#---- Check if it is a FLUSH
suits = [suit for _, suit in cards_retrieved]
is_flush = len(suits) == 1

#---- Check if it is a pair or a triple
# Get the values of the cards
cards = [card for card, _ in cards_retrieved]
# Convert the values of the cards into the Counter object
cards_counter = Counter(cards)
# Calculate how many times occurs each value of the card 
card_counts = list(cards_counter.values())

is_pair = card_counts.count(2) == 1 and card_counts.count(3) == 0
is_triple = card_counts.count(3) == 1 and card_counts.count(2) == 0


#---- Check if it is a pair or a straight
# Create a dictionary of numeric card rakings
# P.s. an ace can be equal to 14 or 1. In most cases, it is equal to 14
# but in the wheel case, a straight hand when a player has Ace, 2, 3, 4, 5,
# the Ace is equal to 1.
card_dict = {
    "ACE": 14, "KING": 13, "QUEEN": 12, "JACK": 11,
    "10": 10, "9": 9, "8": 8, "7": 7, "6": 6,
    "5": 5, "4": 4, "3": 3, "2": 2
}

# Get the card values
# Convert the card values into a set to get only unique values
# Sort the values from the smallest to the largest value
values = sorted({card_dict[c] for c in cards})  # set removes duplicates

# Check if the player got a straight hand
# He got a straight hand if there are five unique values in values object, and
# if the highest value in the list minus the lowest is equal to 4.
# Besides, a player also has a hand if his cards are equal to a wheel (poker term),[2, 3, 4, 5, 14]
is_straight = (
    len(values) == 5 and
    (
        values[4] - values[0] == 4 
        or values == [2, 3, 4, 5, 14]  # Ace low
    )
)


#---- Inform the player if they got straight, flush, triple, or a pair
if is_straight:
    print("Cangratulations you got a straight")
elif is_flush:
    print("Cangratulations you got a flush")
elif is_triple:
    print("Cangratulations you got a triple")
elif is_pair:
    print("Cangratulations you got a pair")
else:
    print("You did not get a flush, a straight, a triple, or a pair hand.")

Cangratulations you got a pair
